<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-данных" data-toc-modified-id="Описание-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание данных</a></span></li><li><span><a href="#Загрузка-библиотек" data-toc-modified-id="Загрузка-библиотек-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка библиотек</a></span></li><li><span><a href="#Загрузка-файла-и-осмотр-данных" data-toc-modified-id="Загрузка-файла-и-осмотр-данных-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Загрузка файла и осмотр данных</a></span></li><li><span><a href="#Preset" data-toc-modified-id="Preset-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Preset</a></span><ul class="toc-item"><li><span><a href="#В-Preset-после-загрузки-базы-данных,-переходим-во-вкладку-SQL-LAB--и-пишем-запрос:" data-toc-modified-id="В-Preset-после-загрузки-базы-данных,-переходим-во-вкладку-SQL-LAB--и-пишем-запрос:-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>В Preset после загрузки базы данных, переходим во вкладку SQL LAB  и пишем запрос:</a></span></li></ul></li></ul></div>

# Формирование запроса в Preset  для построения когортного анализа

## Описание данных

Таблица `nomenk_wine`  содержит сущности:  
- `date` - первый день месяца,  
- `lvl_5` - номенклатура,  
- `card_id` - ключ покупателя,   
- `checks` - количество уникальных чеков. 

## Загрузка библиотек

In [1]:
import pandas as pd
import requests
from urllib.parse import urlencode

In [2]:
from datetime import datetime                               # для преобразования к типу данных 'дата'
import datetime as dt

In [3]:
import seaborn as sns
from matplotlib import pyplot as plt

## Загрузка файла и осмотр данных

In [4]:
# получаем прямую ссылку на файлы
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'


# ссылки на данные с указанием разделителя csv-файла
links_dic = {
    'comandor_11': ['https://disk.yandex.ru/d/A_mPari4YOl8ZA', ';']}

# скачиваем файлы
for csv_filename, csv_link in links_dic.items():
    final_url = base_url + urlencode(dict(public_key=csv_link[0]))
    response = requests.get(final_url)
    download_url = response.json()['href']
    globals()[csv_filename] = pd.read_csv(download_url, sep=csv_link[1])

Взглянем на данные:

In [5]:
comandor_11.head()

,date,lvl_5,card_id,checks
0,2022-01-01,ВИНО АРОМАТИЗИРОВАННОЕ,3338,1
1,2022-01-01,ВИНО АРОМАТИЗИРОВАННОЕ,3988,1
2,2022-01-01,ВИНО АРОМАТИЗИРОВАННОЕ,12237,1
3,2022-01-01,ВИНО АРОМАТИЗИРОВАННОЕ,14576,1
4,2022-01-01,ВИНО АРОМАТИЗИРОВАННОЕ,18316,1


In [6]:
comandor_11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 474168 entries, 0 to 474167
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   date     474168 non-null  object
 1   lvl_5    474168 non-null  object
 2   card_id  474168 non-null  int64 
 3   checks   474168 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 14.5+ MB


Посмотрим на список товаров:

In [7]:
comandor_11.lvl_5.unique()

array(['ВИНО АРОМАТИЗИРОВАННОЕ', 'ВИНО БЕЛОЕ', 'ВИНО БЭГ-ИН-БОКС/ТЕТРАП.',
       'ВИНО КРАСНОЕ', 'ВИНО КРЕПЛЕНОЕ, КАГОР', 'ВИНО ПЛОДОВЫЕ',
       'ВИНО РОЗОВЫЕ', 'ВИСКИ', 'ВОДА 1Л И БОЛЕЕ', 'ВОДА МЕНЕЕ 1 ЛИТРА',
       'ВОДКА ОБЫКНОВЕННАЯ', 'ВОДКА ОСОБАЯ',
       'ГАЗИРОВАННЫЙ ВИННЫЙ НАПИТОК', 'ДИСТИЛЛЯТЫ ПРОЧИЕ',
       'ИМП. КОНЬЯК, БРЕНДИ, АРМАНЬЯК', 'КОНЬЯК, БРЕНДИ РОССИЯ',
       'ЛИКЕРЫ, БАЛЬЗАМЫ', 'НАПИТКИ Б/А 1Л И БОЛЕЕ',
       'НАПИТКИ Б/А МЕНЕЕ 1 Л', 'НАСТОЙКИ', 'ПИВО БОЛЕЕ 1 ЛИТРА',
       'ПИВО ДО 1 ЛИТРА', 'ПРЕДМЕТЫ ДЛЯ КУРЕНИЯ', 'РАЗЛИВНЫЕ НАПИТКИ',
       'СИГАРЕТЫ', 'СИРОПЫ И ФРУКТ. НАПИТКИ', 'СЛАБЫЙ АЛКОГОЛЬ',
       'СОКИ И НЕКТАРЫ 1 Л И МЕНЕЕ', 'СОКИ И НЕКТАРЫ В ИНД УПАКОВКЕ',
       'СОКИ И НЕКТАРЫ СВЫШЕ 1 Л.', 'ШАМПАНСКИЕ И ИГРИСТЫЕ'], dtype=object)

Сумма списка товаров:

In [8]:
comandor_11.lvl_5.nunique()

31

Посмотрим на общее количество покупателей:

In [9]:
comandor_11.card_id.count()

474168

Посмотрим на количество уникальных покупателей:

In [10]:
comandor_11.card_id.nunique()

68549

Приводём дату к формату даты:

In [11]:
comandor_11['date'] = pd.to_datetime(comandor_11['date'].astype(str))
comandor_11 = comandor_11.rename(columns = {'date': 'time'})


Убедимся, что преобразование получилось:

In [12]:
comandor_11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 474168 entries, 0 to 474167
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   time     474168 non-null  datetime64[ns]
 1   lvl_5    474168 non-null  object        
 2   card_id  474168 non-null  int64         
 3   checks   474168 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 14.5+ MB


## Preset

### В Preset после загрузки базы данных, переходим во вкладку SQL LAB  и пишем запрос:

Построим запрос для визуализации кагорт:

SELECT date, start_date, COUNT(card_id) as active_users FROM # формируем основу данных для когорт

(SELECT card_id, min(Date(time)) as start_date  # определяем стартовые даты для покупателей, т.е. когда клиенты впервые
FROM main.comandor_moseeva                      # совершили покупку
GROUP BY card_id
) t1

JOIN 

(SELECT DISTINCT card_id, Date(time) as date     # определяем уникальный покупателей для всего периода датасета
FROM main.comandor_moseeva                       #
) t2

using card_id                                    # объединяем запросы для формирования кагорт

GROUP BY date, start_date                        # делаем группирповку

- Переходим к визуализации- выбираем тип графика Heatmap (тепловую карту)  
- по оси Х отложим `date`  
- по оси Y отложим `start_date` 
- В качестве метрики возьмём значение `active_users` и agdregate  выберем любую фенкцию, которая нам ничего не меняет-например MAX. 

В Preset  не удалось визуализировать, выдаёт ошибки на синтаксис и ругается на тип данных. Данная конструкция должна работать, думаю в Preset  не все так хорошо в типах данных.   В Superset `card_id` был бы: uint32_t, `time` тип данных показывает сразу двух видов: TEXT и DATE